## Reto 5: Automatizando peticiones

### 1. Objetivos:
    - Automatizar la petición de datos a la API de la NASA por fechas.
 
---
    
### 2. Desarrollo:

### a) Automatizando peticiones

En el primer Reto realizaste una petición a la API de la NASA para pedir 5 datos de la hoja #100. Ahora, vamos a automatizar el proceso de realizar peticiones a la API para obtener una cantidad bastante mayor de datos.

Vamos a obtener ahora los datos por fechas, así que tendrás que ir a la página de la [api de la NASA](https://api.nasa.gov/), para revisar cómo es posible realizar peticiones por fecha. Queremos obtener los **dos primeros meses (Enero y Febrero) del año 1995**. Observa que la API sólo permite peticiones por fecha en rangos de 7 días.

Tu reto tiene los siguientes pasos:

1. Revisa la documentación de la API de la NASA para entender cómo realizar peticiones por fecha.
2. Realiza una primera petición de prueba para entender el formato de los datos que obtienes de regreso (cómo extraemos los datos que necesitamos y qué estructura tienen).
2. Escribe el código necesario para automatizar las peticiones a la API y obtener los meses de Enero y Febrero del año 1995.
3. Almacena los datos de cada petición y luego usa esos datos para crear `DataFrames`.
4. Concatena verticalmente tus `DataFrames` para obtener un `DataFrame` final que contenga todos los datos de tus peticiones. Cada fila tiene que corresponder a un objeto espacial.
5. Guarda tu `DataFrame` con el nombre de `near_earth_objects-january_february_1995-raw.csv`.

> **Nota**: En este momento no te preocupes por explorar o limpiar tu dataset. Eso lo haremos en la siguiente sesión. Lo que sí tienes que asegurarte es de normalizar tus datos antes de convertirlos en `DataFrame`.

¡Mucha suerte!

Importamos las librerias necesarias

In [ ]:
import pandas as pd 
import requests
from pprint import pprint

Definimos el endpoint e ingresamos los parametros 

In [ ]:
payload = {'api_key':'QAXLpeREJyy8YUciiVBq0v42tuc0Hjn6Izb6DiZj','start_date':'1995-01-01','end_date':'1995-01-01'}
endpoint = "https://api.nasa.gov/neo/rest/v1/feed"

Realizamos una primera petición de prueba

In [ ]:
r = requests.get(endpoint,payload)
r.status_code

200

In [ ]:
json = r.json()

data = json['near_earth_objects']

normalized = pd.json_normalize(data,record_path=['1995-01-01'])
close_approach_data = pd.json_normalize(data,record_path=['1995-01-01','close_approach_data'],record_prefix='close_approach_data.')

objetos1 = pd.DataFrame.from_dict(normalized)
objetos2 = pd.DataFrame.from_dict(close_approach_data)

objetos1.head()

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,3703080,3703080,(2014 YW34),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3703080,21.70,True,"[{'close_approach_date': '1995-01-01', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3703080?ap...,0.121494,0.271669,121.494041,271.668934,0.075493,0.168807,398.602509,891.302306
1,3723955,3723955,(2015 NK3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3723955,21.30,False,"[{'close_approach_date': '1995-01-01', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3723955?ap...,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620,1071.581063
2,3792437,3792437,(2017 XR61),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3792437,23.10,False,"[{'close_approach_date': '1995-01-01', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/3792437?ap...,0.063761,0.142574,63.760979,142.573883,0.039619,0.088591,209.189570,467.762099
3,54105518,54105518,(2021 AW5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54105518,26.54,False,"[{'close_approach_date': '1995-01-01', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/54105518?a...,0.013078,0.029244,13.078411,29.244216,0.008127,0.018172,42.908173,95.945593


In [ ]:
objetos2.head()

,close_approach_data.close_approach_date,close_approach_data.close_approach_date_full,close_approach_data.epoch_date_close_approach,close_approach_data.orbiting_body,close_approach_data.relative_velocity.kilometers_per_second,close_approach_data.relative_velocity.kilometers_per_hour,close_approach_data.relative_velocity.miles_per_hour,close_approach_data.miss_distance.astronomical,close_approach_data.miss_distance.lunar,close_approach_data.miss_distance.kilometers,close_approach_data.miss_distance.miles
0,1995-01-01,1995-Jan-01 10:08,788954880000,Earth,6.1687775767,22207.5992761981,13798.9385613687,0.4212616856,163.8707956984,63019850.878369672,39158719.5218665936
1,1995-01-01,1995-Jan-01 03:35,788931300000,Earth,18.1141417391,65210.9102609105,40519.5236562863,0.3829859177,148.9815219853,57293877.527915299,35600764.6664531662
2,1995-01-01,1995-Jan-01 14:07,788969220000,Earth,16.5197198656,59470.9915161269,36952.9613673401,0.2635480219,102.5201805191,39426222.718953353,24498318.7954201514
3,1995-01-01,1995-Jan-01 11:19,788959140000,Earth,19.3765128633,69755.4463077572,43343.3216238395,0.4132372129,160.7492758181,61819406.854576523,38412798.1943130974


In [ ]:
df_completo = pd.concat([objetos1,objetos2],axis=1)
df_completo = df_completo.drop(columns=['close_approach_data'])
df_completo

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max,close_approach_data.close_approach_date,close_approach_data.close_approach_date_full,close_approach_data.epoch_date_close_approach,close_approach_data.orbiting_body,close_approach_data.relative_velocity.kilometers_per_second,close_approach_data.relative_velocity.kilometers_per_hour,close_approach_data.relative_velocity.miles_per_hour,close_approach_data.miss_distance.astronomical,close_approach_data.miss_distance.lunar,close_approach_data.miss_distance.kilometers,close_approach_data.miss_distance.miles
0,3703080,3703080,(2014 YW34),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3703080,21.70,True,False,http://www.neowsapp.com/rest/v1/neo/3703080?ap...,0.121494,0.271669,121.494041,271.668934,0.075493,0.168807,398.602509,891.302306,1995-01-01,1995-Jan-01 10:08,788954880000,Earth,6.1687775767,22207.5992761981,13798.9385613687,0.4212616856,163.8707956984,63019850.878369672,39158719.5218665936
1,3723955,3723955,(2015 NK3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3723955,21.30,False,False,http://www.neowsapp.com/rest/v1/neo/3723955?ap...,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620,1071.581063,1995-01-01,1995-Jan-01 03:35,788931300000,Earth,18.1141417391,65210.9102609105,40519.5236562863,0.3829859177,148.9815219853,57293877.527915299,35600764.6664531662
2,3792437,3792437,(2017 XR61),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3792437,23.10,False,False,http://www.neowsapp.com/rest/v1/neo/3792437?ap...,0.063761,0.142574,63.760979,142.573883,0.039619,0.088591,209.189570,467.762099,1995-01-01,1995-Jan-01 14:07,788969220000,Earth,16.5197198656,59470.9915161269,36952.9613673401,0.2635480219,102.5201805191,39426222.718953353,24498318.7954201514
3,54105518,54105518,(2021 AW5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54105518,26.54,False,False,http://www.neowsapp.com/rest/v1/neo/54105518?a...,0.013078,0.029244,13.078411,29.244216,0.008127,0.018172,42.908173,95.945593,1995-01-01,1995-Jan-01 11:19,788959140000,Earth,19.3765128633,69755.4463077572,43343.3216238395,0.4132372129,160.7492758181,61819406.854576523,38412798.1943130974


In [ ]:
payload = {'api_key':'QAXLpeREJyy8YUciiVBq0v42tuc0Hjn6Izb6DiZj','start_date':'1995-01-01','end_date':'1995-01-01'}
endpoint = "https://api.nasa.gov/neo/rest/v1/feed"

dict_datos = {}
for i in range(0,59):
  try:
    r = requests.get(endpoint, params=payload)

    if r.status_code == 200:
      json = r.json()

      data = json['near_earth_objects']
      dict_datos[i] = data

      new_link = json['links']['next']
      endpoint = new_link
    
  except:
    continue

In [ ]:
len(dict_datos)

59

In [ ]:
lista_fechas = pd.date_range(start="1995-01-01",end="1995-02-28").strftime('%Y-%m-%d')

len(lista_fechas)

59

In [ ]:
lista_fechas[0]

'1995-01-01'

In [ ]:
for par in dict_datos:
  normalizados = pd.json_normalize(dict_datos[par],record_path=[lista_fechas[par])
  df = pd.DataFrame.from_dict(normalizados)
  dict_datos[par] = df

In [ ]:
lista_de_dataframes = []

for key in dict_datos:
    lista_de_dataframes.append(dict_datos[key])

In [ ]:
df_completo = pd.concat(lista_de_dataframes, axis=0).reset_index(drop=True)

In [ ]:
df_completo

,close_approach_date,close_approach_date_full,epoch_date_close_approach,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles
0,1995-01-01,1995-Jan-01 10:08,788954880000,Earth,6.1687775767,22207.5992761981,13798.9385613687,0.4212616856,163.8707956984,63019850.878369672,39158719.5218665936
1,1995-01-01,1995-Jan-01 03:35,788931300000,Earth,18.1141417391,65210.9102609105,40519.5236562863,0.3829859177,148.9815219853,57293877.527915299,35600764.6664531662
2,1995-01-01,1995-Jan-01 14:07,788969220000,Earth,16.5197198656,59470.9915161269,36952.9613673401,0.2635480219,102.5201805191,39426222.718953353,24498318.7954201514
3,1995-01-01,1995-Jan-01 11:19,788959140000,Earth,19.3765128633,69755.4463077572,43343.3216238395,0.4132372129,160.7492758181,61819406.854576523,38412798.1943130974
4,1995-01-02,1995-Jan-02 15:31,789060660000,Earth,14.9704950984,53893.7823541644,33487.500485564,0.0732050147,28.4767507183,10951314.272438689,6804831.1446803482
...,...,...,...,...,...,...,...,...,...,...,...
390,1995-02-28,1995-Feb-28 23:19,794013540000,Earth,3.4537107738,12433.3587857208,7725.6056299385,0.284938158,110.840943462,42626141.51852346,26486656.137045348
391,1995-02-28,1995-Feb-28 05:02,793947720000,Earth,18.3574799623,66086.9278641936,41063.8469276152,0.4218178172,164.0871308908,63103046.981169364,39210415.1829866632
392,1995-02-28,1995-Feb-28 17:27,793992420000,Earth,11.6584316545,41970.3539560484,26078.7457678569,0.1940774472,75.4961269608,29033572.716157464,18040625.5308984432
393,1995-02-28,1995-Feb-28 10:36,793967760000,Earth,5.3934832875,19416.5398351639,12064.6827659106,0.0691001415,26.8799550435,10337233.985098605,6423259.347846549


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#df_completo.to_csv('../../Datasets/near_earth_objects-january_february_1995-raw.csv')